In [1]:
getwd()
setwd("/liulab/galib/dlbcl_manuscript/")
library(tidyverse)
library(Seurat)
library(here)
library(harmony)
library(viridis)
library(RColorBrewer)
library(Polychrome)
PurpleAndYellow()
library(ComplexHeatmap)
library(devtools)
library(presto)
library(dplyr)
library(ggplot2)
library(ggpubr)

[1] "/liulab/galib/dlbcl_manuscript/scripts/10_infercnv"

Warning message:
“package ‘tidyverse’ was built under R version 4.1.3”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.0      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.1 

Warning message:
“package ‘tidyr’ was built under R version 4.1.2”
Warning message:
“package ‘readr’ was built under R version 4.1.2”
Warning message:
“package ‘forcats’ was built under R version 4.1.3”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching SeuratObject

Attaching sp

here() starts at /liulab/galib/dlbcl_manuscript

Warning message:
“package ‘harmony’ was built under R version 4.1.3”
Loading required package: Rcpp

Warning message:
“package ‘Rcpp’ was built under R version 4.1.2”
Loading required package: viridisLite

Warning message:
“package ‘RColorBrewer

[1] "#FF00FF" "#F400F4" "#EA00EA" "#DF00DF" "#D500D5" "#CA00CA" "#BF00BF"
 [8] "#B500B5" "#AA00AA" "#9F009F" "#950095" "#8A008A" "#800080" "#750075"
[15] "#6A006A" "#600060" "#550055" "#4A004A" "#400040" "#350035" "#2B002B"
[22] "#200020" "#150015" "#0B000B" "#000000" "#000000" "#0B0B00" "#151500"
[29] "#202000" "#2B2B00" "#353500" "#404000" "#4A4A00" "#555500" "#606000"
[36] "#6A6A00" "#757500" "#808000" "#8A8A00" "#959500" "#9F9F00" "#AAAA00"
[43] "#B5B500" "#BFBF00" "#CACA00" "#D4D400" "#DFDF00" "#EAEA00" "#F4F400"
[50] "#FFFF00"

Warning message:
“package ‘ComplexHeatmap’ was built under R version 4.1.3”
Loading required package: grid

ComplexHeatmap version 2.10.0
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite:
Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
  genomic data. Bioinformatics 2016.

The new InteractiveComplexHeatmap package can directly export static 
complex heatmaps into an interactive Shiny app with zero effort. Have a try!

This message can be suppressed by:
  suppressPackageStartupMessages(library(ComplexHeatmap))


Loading required package: usethis

Warning message:
“package ‘presto’ was built under R version 4.1.3”
Loading required package: data.table


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The

In [2]:
B_cell<- readRDS("./data/objects/B_cell_final.rds")

In [3]:
infercnv_pos_cells<- read.table('data/infercnv/10_B_cell_inferCNV_pos_PC5.tsv', header = FALSE, sep = '\t')  %>% pull('V1')

In [6]:
length(infercnv_pos_cells)

[1] 42331

In [7]:
all_poolid_sick<- B_cell@meta.data  %>% filter(age == 'sick')  %>% 
    count(pool_id, sample_id, genotype, age)  %>% arrange(age, genotype)

all_poolid_sick$genotype_new<- gsub(all_poolid_sick$genotype, pattern = '/[-+]', replacement = "")  %>% 
    gsub(pattern = ';', replacement = "_")  %>% gsub(pattern = '-', replacement = "")

In [8]:
inpath = '/liulab/galib/mouse_scRNAseq_margaret/data/infercnv/inferCNV_out_new/'

In [12]:
# with cluster annotation
for (i in c(1:nrow(all_poolid_sick))){
    
    poolid = all_poolid_sick$pool_id[i]
    genotype = all_poolid_sick$genotype[i]
    age = all_poolid_sick$age[i]
    sampleid = all_poolid_sick$sample_id[i]
    
    genotype_new<- gsub(genotype, pattern = '/[-+]', replacement = "")  %>% 
        gsub(pattern = ';', replacement = "_")  %>% gsub(pattern = '-', replacement = "")

    message('Reading ', paste0(inpath,poolid, '_newsick/run.final.infercnv_obj ...'))
    
    infercnv_obj<- readRDS(paste0(inpath, poolid, '_newsick/run.final.infercnv_obj'))
    gene_position = infercnv_obj@gene_order
    mtx_obs<- infercnv_obj@expr.data[,infercnv_obj@observation_grouped_cell_indices[[genotype]]]
    mtx_obs = t(mtx_obs)
    mtx_ref<- infercnv_obj@expr.data[,infercnv_obj@reference_grouped_cell_indices[['WT']]]
    mtx_ref = t(mtx_ref)
    
    # set heatmap row annotation
    row_annotation = B_cell@meta.data  %>% filter(pool_id==poolid)  %>% select(new_annotation)  %>% mutate(infercnv = 'infercnv-neg')
    row_annotation[rownames(row_annotation) %in% infercnv_pos_cells, 'infercnv'] = 'infercnv-pos'
    row_annotation =  row_annotation %>% arrange(infercnv, new_annotation)

    new_annotation_hex= B_cell@meta.data  %>% 
                                filter(pool_id==poolid)  %>% 
                                select(new_annotation, HEX_code)  %>% 
                                distinct()  %>% arrange(new_annotation)

    new_annotation_colors = new_annotation_hex$HEX_code
    names(new_annotation_colors) = new_annotation_hex$new_annotation

    ra <- rowAnnotation(clusterID = row_annotation$new_annotation, infercnv = row_annotation$infercnv,
                        col = list(infercnv = c("infercnv-pos" = '#FF7F00', "infercnv-neg"= '#984EA3'),
                                   clusterID = new_annotation_colors))

    cnv_col_fun<- circlize::colorRamp2(c(1 - max(abs(mtx_obs - 1)), 1, 1 + max(abs(mtx_obs - 1))), c("blue", "white", "red"))
    cnv_col_fun<- circlize::colorRamp2(c(0.8, 1, 1.2), c("blue", "white", "red"))
    ht = Heatmap(mtx_obs[rownames(row_annotation),],
            name = paste0(poolid, '_', sampleid), 
            border_gp = gpar(col = "black"),
            column_split = factor(gene_position[colnames(mtx_obs), 'chr'], levels = unique(gene_position[['chr']])),
            column_title_rot = 90,
            column_title_gp = gpar(fontsize = 6),
            column_gap = unit(0.1, "mm"),
            cluster_rows = FALSE, 
            cluster_columns = FALSE, 
            show_row_names = FALSE,
            show_column_names = FALSE, 
            right_annotation = ra,
            use_raster = TRUE,
            show_heatmap_legend = TRUE,
            col = cnv_col_fun)
    pdf(paste0('./results/figures/12_inferCNV_labeled_heatmap_', poolid, '_', sampleid, '_', genotype_new, '_', age, '_with_cluster.pdf'), width = 10, height = 6)
    draw(ht)
    dev.off()
}

Reading /liulab/galib/mouse_scRNAseq_margaret/data/infercnv/inferCNV_out_new/Pool105_1_newsick/run.final.infercnv_obj ...

'magick' package is suggested to install to give better rasterization.

Set `ht_opt$message = FALSE` to turn off this message.

Reading /liulab/galib/mouse_scRNAseq_margaret/data/infercnv/inferCNV_out_new/Pool105_2_newsick/run.final.infercnv_obj ...

'magick' package is suggested to install to give better rasterization.

Set `ht_opt$message = FALSE` to turn off this message.

Reading /liulab/galib/mouse_scRNAseq_margaret/data/infercnv/inferCNV_out_new/Pool105_3_newsick/run.final.infercnv_obj ...

'magick' package is suggested to install to give better rasterization.

Set `ht_opt$message = FALSE` to turn off this message.

Reading /liulab/galib/mouse_scRNAseq_margaret/data/infercnv/inferCNV_out_new/Pool105_4_newsick/run.final.infercnv_obj ...

'magick' package is suggested to install to give better rasterization.

Set `ht_opt$message = FALSE` to turn off this messag